# Retrieval-Augmented Generation (RAG)

In [17]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [18]:
# We populated the RAG with the data from the data/calories.csv file in
# the rag_setup.ipynb notebook

chroma_client = chromadb.PersistentClient("../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")
nutrition_qna = chroma_client.get_collection(name="nutrition_qna")

In [16]:
results = nutrition_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

result2 = nutrition_qna.query(query_texts=["pregnacy"], n_results=2)
for i2, doc2 in enumerate(result2["documents"][0]):
    print(sorted(result2["metadatas"][0][i2].items()))
    print(doc2)
    print("\n")	


[('calories_per_100g', 89.0), ('food_category', 'fruits'), ('food_item', 'banana'), ('keywords', 'banana_fruits'), ('kj_per_100g', 374.0), ('serving_info', '100g')]
Food: Banana
        Category: Fruits
        Nutritional Information:
        - Calories: 89 per 100g
        - Energy: 374 kJ per 100g
        - Serving size reference: 100g

        This is a fruits food item that provides 89 calories per 100 grams.


[('calories_per_100g', 50.0), ('food_category', '(fruit)juices'), ('food_item', 'banana juice'), ('keywords', 'banana_juice_(fruit)juices'), ('kj_per_100g', 210.0), ('serving_info', '100ml')]
Food: Banana Juice
        Category: (Fruit)Juices
        Nutritional Information:
        - Calories: 50 per 100g
        - Energy: 210 kJ per 100g
        - Serving size reference: 100ml

        This is a (fruit)juices food item that provides 50 calories per 100 grams.


[('is_pregnancy', True)]
Question: What factors can contribute to a high risk pregnaancy?
        Answer: Factor

In [19]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

In [26]:
@function_tool
def pregnacy_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up pregnacy information for specific time items.

    Args:
        query: The time item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the pregnacy information.
    """

    results = nutrition_qna.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No pregnacy information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
#        metadata = results["metadatas"][0][i]
#        qna_item = metadata["qna_item"].title()
#        calories = metadata["calories_per_100g"]
#        category = metadata["food_category"].title()

        formatted_results.append(
            f"{doc} :pregnacy"
        )

    return "pregnacy Information:\n" + "\n".join(formatted_results)

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `calorie_lookup_tool` function_

In [ ]:
# calorie_lookup_tool('bananas')
# pragnacy_lookup_tool('symptoms')

KeyError: 'qna_item'

In [28]:
calorie_agent = Agent(
    name="Nutrition Pregnacy Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    """,
    tools=[calorie_lookup_tool, pregnacy_lookup_tool],
)

In [30]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result.final_output)
    result = await Runner.run(
        calorie_agent,
        "what happens with the body during pregnacy?",
    )
    print(result.final_output)

- Calories per 100 g: banana 89 kcal; apple 52 kcal.
- Estimated total for a medium banana (~118 g) + a medium apple (~182 g): about 200 kcal. 
- If sizes differ, recalibrate using the per-100 g values (kcal = weight_g × kcal_per_100g / 100).
Here’s a concise overview of what happens in the body during pregnancy:

- Hormonal changes: Estrogen and progesterone rise, supporting the pregnancy but also causing symptoms like nausea, fatigue, mood changes, and breast changes.
- Blood and circulation: Blood volume increases to supply the fetus; this can lead to lightheadedness, edema, and higher heart rate.
- Placenta and baby: The placenta forms and grows, delivering oxygen and nutrients to the fetus and removing waste.
- Metabolism: Basal metabolic rate rises. Insulin resistance can increase, which, in some pregnancies, raises blood sugar risk (gestational diabetes).
- Weight and organs: Uterus enlarges, abdominal organs get displaced, lungs may feel compressed, and joints may loosen (due t